[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pdf-tools/components-code-sample-hub/blob/main/jupyter/pdftools_sdk/pdftools_sdk_swisscom_sig_srv_add_timestamp.ipynb)

In [ ]:
%pip install https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/productkits/PDFSDK/latest/pdftools_sdk-latest.tar.gz
%pip install ipython

# Add a document time-stamp to a PDF using the Swisscom Signing Service
Add a trusted document time-stamp to a PDF and confirm
that the signed document has not been altered. This type
of signature proves that the document existed at a
specific time and ensures its integrity.

In [ ]:
import io
from pdftools_sdk.pdf import Document
from pdftools_sdk.sign import Signer
from pdftools_sdk.crypto.providers.swisscom_sig_srv import Session
from pdftools_sdk.http_client_handler import HttpClientHandler

In [ ]:
# Download a file from a given URL and save it to the local system
def prepare_file(url: str, path: str):
    import requests
    response = requests.get(url)
    response.raise_for_status()

    with open(path, 'wb') as f:
        f.write(response.content)

In [ ]:
# Set input arguments
identity = 'INSERT-IDENTITY'  # Placeholder for Swisscom identity
input_url = 'https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/samples/testfiles/InvoiceNone.pdf'
input_path = 'InvoiceNone.pdf'
prepare_file(input_url, input_path)
output_path = 'TimestampedSwisscom.pdf'

In [ ]:
def add_timestamp(session: Session, identity: str, input_path: str, output_path: str):
    # Create timestamp configuration
    timestamp = session.create_timestamp(identity)

    # Open input document
    with io.FileIO(input_path, 'rb') as input_stream:
        with Document.open(input_stream) as input_document:
            # Create stream for output file
            with io.FileIO(output_path, 'wb+') as output_stream:
                # Add the document timestamp
                signer = Signer()
                signer.add_timestamp(input_document, timestamp, output_stream)

In [ ]:
try:
    # By default, a test license key is active. In this case, a watermark is added to the output. 
    # If you have a license key, please uncomment the following call and set the license key.
    # from pdftools_sdk.sdk import Sdk
    # Sdk.initialize("<PDFSDK,V1,MGAASQD6L2JMQHL54PK08RQX8GG4SS0M8DAHVPH0VMP3NB8R9DUK>")

    # Optional: Set your proxy configuration
    # Sdk.set_proxy("http://myproxy:8080")
    
    # Configure the SSL client certificate to connect to the service
    http_client_handler = HttpClientHandler()
    with io.FileIO("C:/path/to/clientcert.p12", 'rb') as cert_stream:
        http_client_handler.set_client_certificate(cert_stream, "***insert password***")
    
        # Connect to the Swisscom Signing Service
        with Session("https://ais.swisscom.com", http_client_handler) as session:
            # Add a document timestamp to a PDF
            add_timestamp(session, identity, input_path, output_path)

    print(f"Successfully created file {output_path}")
except Exception as e:
    print(f"An error occurred: {e}")